In [ ]:
import json
import pandas as pd
with open("outputs/buckets_07-29_charsim17.json","r") as file:
  buckets = json.load(file)
bucket_map={}
for name in buckets:
  for action in buckets[name]:
    bucket_map[action]=name
common_bucket_names = [name for name in buckets.keys() if len(buckets[name])>=10]
common_bucket_names_inv={name:i for i,name in enumerate(common_bucket_names)}
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df_by_term=[pd.read_csv(ds) for ds in datasets]
for i,df in enumerate(history_df_by_term):
  df["term"]=i+111
history_df=pd.concat(history_df_by_term)
# with open("./outputs/bucket_names_output.jsonl", "r") as file:
#   for line in file:
#     if line.strip():  # Skip empty lines
#       response = json.loads(line)
#       llm_buckets.append(response["response"]["body"]["output"][0]["content"][0]["text"])
      
# history_df["llm_bucket"] = history_df["bucket"].map(dict(zip(bucket_names, llm_buckets)))

# Add next bill_id column to compare
history_df["bucket"]=history_df["action"].apply(lambda action:bucket_map[action])
bills={bill_id:group for (bill_id,group) in history_df.groupby("bill_id")}
bill_ids = list(bills.keys())

MIN_TERM=min(history_df["term"])
MAX_TERM=max(history_df["term"])
N_TERMS=MAX_TERM-MIN_TERM+1


In [ ]:
print(len(buckets.keys()))

In [ ]:
print(len(common_bucket_names))

In [ ]:
import numpy as np

alpha=2/3
data=[]

bill_id=history_df.iloc[0]["bill_id"]
predecessors_vec=np.zeros(len(common_bucket_names))
predecessor_vec=np.zeros(len(common_bucket_names))
one_hot_curr = np.zeros(len(common_bucket_names))
inputs=[]
outputs=[]
n_predecessors_prev=0
for i,row in history_df.iterrows():
  one_hot_prev = np.array(one_hot_curr)
  one_hot_curr = np.zeros(len(common_bucket_names))
  if row.bucket in common_bucket_names_inv:
    one_hot_curr[common_bucket_names_inv[row.bucket]]=1

  # if sum(one_hot_prev)!=1 and sum(one_hot_prev)!=0:
  #   print("non-0/1 sum",i)
  # n_predecessors=np.sum(predecessors_vec)
  # if n_predecessors!=0 and n_predecessors!=n_predecessors_prev+1 and n_predecessors!=n_predecessors_prev:
  #   print(n_predecessors,n_predecessors_prev,i)
  # n_predecessors_prev=n_predecessors
  if row["bill_id"]!=bill_id:
    bill_id=row["bill_id"]
    one_hot_prev=np.zeros(len(common_bucket_names))
    predecessors_vec=np.zeros(len(common_bucket_names))
    predecessor_vec=np.zeros(len(common_bucket_names))
  predecessor_vec=(1-alpha)*one_hot_prev+alpha*np.array(predecessor_vec)
  predecessors_vec=np.array(predecessors_vec)+one_hot_prev
  one_hot_term=np.zeros(N_TERMS)
  one_hot_term[row.term-MIN_TERM]=1
  # input_vec = np.concat([predecessor_vec,predecessors_vec,one_hot_term])
  entry={"predecessor":predecessor_vec,"predecessors":predecessors_vec,"term":one_hot_term,"output_vec":one_hot_curr,"output":common_bucket_names_inv[row.bucket] if row.bucket in common_bucket_names_inv else -1}
  # print(entry)
  data.append(entry)

# print(data[0])
# inputs=np.stack(inputs)
# outputs=np.stack(outputs)
# np.savetxt("outputs/vectors/07-29_charsim17/input",inputs[0])
# np.savetxt("outputs/vectors/07-29_charsim17/output",outputs)


In [ ]:
print(data[0])

In [ ]:
np.savez_compressed("outputs/prediction_vecs_07-29_charsim17.npz",data)

In [ ]:
import numpy as np
data = np.load("outputs/prediction_vecs_07-29_charsim17.npz",allow_pickle=True)["arr_0"]
display(data[0])
# display(outs)

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class ActionDataset(Dataset):
  def __init__(self,path):
    self.data = np.load(path,allow_pickle=True)["arr_0"]
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    entry = self.data[idx]
    input = np.concatenate([entry["predecessor"],entry["predecessors"],entry["term"]])
    return input,entry["output_vec"]

ds = ActionDataset("outputs/prediction_vecs_07-29_charsim17.npz")
# print(len(ds))
train_dataset,test_dataset,val_dataset = torch.utils.data.random_split(ds,[0.6,0.2,0.2])

In [ ]:
def print_arr(arr):
  for line in arr:
    print([float(x) for x in line])

In [ ]:
import torch
import os
from torch.utils.data import DataLoader

# class ActionModel(torch.nn.Module):
#   def __init__(self,input_len,output_len):
#     self.linear= torch.nn.Linear
#     self.input_len=input_len
#     self.output_len = output_len
#   def forward(self,input):
# class ScalarMultiply(torch.nn.Module):
#     """
#     A PyTorch module that multiplies input by a learnable scalar parameter.
    
#     Args:
#         initial_value (float): Initial value for the scalar parameter. Default: 1.0
#         requires_grad (bool): Whether the parameter should be trainable. Default: True
#     """
    
#     def __init__(self, initial_value=1.0, requires_grad=True):
#         super(ScalarMultiply, self).__init__()
#         self.scalar = torch.nn.Parameter(
#             torch.tensor(initial_value, dtype=torch.float32), 
#             requires_grad=requires_grad
#         )
    
#     def forward(self, x):
#         """
#         Forward pass: multiply input by the learnable scalar.
        
#         Args:
#             x (torch.Tensor): Input tensor of any shape
            
#         Returns:
#             torch.Tensor: Input multiplied by the scalar, same shape as input
#         """
#         return x * self.scalar
    
#     def extra_repr(self):
#         """String representation for print/debugging"""
#         return f'scalar={self.scalar.item():.4f}'



# hyperparam_sets=[{"folder":"outputs/models/07-30_lr1e-4beta.01"}]
# for hyperparams in hyperparam_sets:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)


folder="outputs/models/07-30_lr1e-4_beta10"
model = torch.nn.Linear(len(common_bucket_names)*2+N_TERMS,len(common_bucket_names))
# model.load_state_dict(torch.load("outputs/models/07-30_lr1e-5_beta.1/epoch70.pt"))
optim = torch.optim.Adam(model.parameters(),lr=1e-5)
pred_loss_fn = torch.nn.CrossEntropyLoss()
beta=10
epsilon=1e-8
# folder = hyperparams["folder"]
if not os.path.isdir(folder):
    os.mkdir(folder)

def reg_loss_fn(weights):
   return torch.sum(torch.abs(weights))
  #  return torch.sum(torch.sqrt(torch.abs(weights)+epsilon))
for epoch in range(100):
  for i,(input,output) in enumerate(train_loader):
    optim.zero_grad()
    input=input.float()
    output=output.float()
    pred = model(input)
    pred_loss = pred_loss_fn(pred,output)
    reg_loss = reg_loss_fn(model.weight)
    # print("pred loss:",pred_loss)
    # print("reg loss:",reg_loss)
    loss = pred_loss+beta*reg_loss
    loss.backward()
    optim.step()
  print(f"finished epoch {epoch}")
  
  torch.save(model.state_dict(),folder+f"/epoch{epoch}.pt")
  
  with torch.no_grad():
    val_pred_loss=torch.scalar_tensor(0)
    val_reg_loss=reg_loss_fn(model.weight)
    for input,output in val_loader:
      input=input.float()
      output=output.float()
      pred = model(input)
      # for line in output:
      #   for i,x in enumerate(line):
      #     if x>=0.9:
      #       print(i)
      #   print(torch.sum(line))
      # break
      # print_arr(output.numpy())
      # print("sum:",torch.sum(output))
      val_pred_loss+=pred_loss_fn(pred,output)
    val_pred_loss/=len(val_loader)
    with open(folder+f"/losses_epoch{epoch}","w") as file:
        file.write("val pred loss: "+str(float(val_pred_loss))+". val ridge loss: "+str(float(val_reg_loss)))
    print("val pred loss:",val_pred_loss,"val reg loss:",val_reg_loss)



In [ ]:

model = torch.nn.Linear(len(common_bucket_names)*2+N_TERMS,len(common_bucket_names))
# model.load_state_dict(torch.load("outputs/models/07-30_lr1e-5_beta.1/epoch70.pt"))
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)
with torch.no_grad():
  val_pred_loss=torch.scalar_tensor(0)
  val_ridge_loss=torch.sum(torch.abs(model[0].weight))
  for input,output in val_loader:
    input=input.float()
    output=output.float()
    pred = model(input)
    for line in pred:
      print(list(line))
      # for i,x in enumerate(line):
      #   if x>=0.1:
      #     print(i,float(x),end="")
      # print()
      print(torch.sum(line))
    # break
    # print_arr(output.numpy())
    # print("sum:",torch.sum(output))
  # val_pred_loss/=len(val_loader)
  print("val pred loss:",val_pred_loss,"val ridge loss:",val_ridge_loss)

In [ ]:
display(buckets["H.AMDT.1 Amendment (A001) offered by Mr. Becerra. (consideration: CR H6; text: CR H6)Amendment nominated Democrat candidates for Officers of the House."])


In [ ]:
from collections import defaultdict
import torch
model = torch.nn.Linear(len(common_bucket_names)*2+N_TERMS,len(common_bucket_names))
model.load_state_dict(torch.load("outputs/models/07-30_lr1e-5_beta1/epoch79.pt"))
weights = model.weight
weights=weights.detach().numpy()
print(weights.shape)
pred_connections=[]
preds_connections=[]
term_connections=[]
# term_params=[dict() for i in range(N_TERMS)]
for i,bucket1 in enumerate(common_bucket_names):
  for j,bucket2 in enumerate(common_bucket_names):
    if bucket1==bucket2:
      continue
    pred_connections.append((float(weights[j][i]),bucket1,bucket2))
    preds_connections.append(( float(weights[j][i+len(common_bucket_names)]),bucket1,bucket2))
for term_i in range(N_TERMS):
  for j,bucket2 in enumerate(common_bucket_names):
    term_connections.append(( float(weights[j][term_i+2*len(common_bucket_names)]),bucket1,bucket2))

pred_connections.sort(reverse=True, key = lambda x:abs(x[0]))
preds_connections.sort(reverse=True,key = lambda x:abs(x[0]))
term_connections.sort(reverse=True,key = lambda x:abs(x[0]))
for connection in pred_connections[:100]:
  print(connection[0],connection[1])
# for connection in preds_connections[:500]:
#   print(connection[0], connection[1])
# for connection in preds_connections[-500:]:
#   print(connection[0], connection[1])
# display("predecessor:",pred_connections)
# display("predecessors:",preds_connections)
# display("terms:",term_connections)

In [ ]:
pred_connections=[]
preds_connections=[]
for bucket1 in common_bucket_names:
  for bucket2 in common_bucket_names:
    pred_connections.append((predecessor_params[bucket1][bucket2],bucket1 +" -> "+bucket2))
    preds_connections.append((predecessors_params[bucket1][bucket2],bucket1 +" -> "+bucket2))
for term_i in range(N_TERMS):
  for bucket2 in common_bucket_names:
    pred_connections.append((predecessor_params[bucket1][bucket2],bucket1 +" -> "+bucket2))
    preds_connections.append((predecessors_params[bucket1][bucket2],bucket1 +" -> "+bucket2))

In [ ]:
# Sklearn
import sklearn
import copy
import random
model = sklearn.linear_model.LogisticRegression(penalty="l1",solver="saga",multi_class="multinomial")

scaler = sklearn.preprocessing.StandardScaler()
data_shuffled = copy.deepcopy(data)
random.shuffle(data_shuffled)
inputs = [np.concatenate([e["predecessors"],e["predecessor"],e["term"]]) for e in data_shuffled]
outputs = [e["output"] for e in data_shuffled]
inputs= scaler.fit_transform(inputs)
divider = int(len(data)*0.8)
train_inputs,test_inputs = inputs[:divider], data_shuffled[divider:]
train_outputs,test_outputs = outputs[:divider], outputs[divider:]
model.fit(train_inputs,train_outputs)
test_pred = model.predict_proba(test_inputs) 